# V1: SCF optimization with VAMPyR

## Hydrogen atom

We intend to solve the Schrödinger equation for the Hydrogen atom

\begin{equation*}
  \left[\hat{T} + \hat{V}\right]\phi = E\phi
\end{equation*}

where the potential operator contains only the nuclear attraction

\begin{equation*}
  \hat{V}(r) = \frac{-Z}{|R-r|}
\end{equation*}

where $Z=1$ and $R=(0,0,0)$ are the nuclear charge and position, respectively.

### Solution using multiwavelets
In order to solve this equation using multiwavelets we reformulate it in an integral form:

\begin{equation}
  \phi = -2\hat{G}_{\mu}\hat{V}\phi
\end{equation}

and iterate until convergence. $\hat{G}_\mu$ is the Bound-State Helmholtz (BSH) integral operator, whose kernel is defined as

\begin{equation}
  G_\mu(r - r') = \frac{\exp(-\mu |r - r'|)}{4\pi |r - r'|}
\end{equation}

and $\mu=\sqrt{-2E}$ takes the place of the energy in the original Schrödinger equation. For the Hydrogen atom this energy is known (E=-0.5 a.u.) so we can use the "exact" BSH operator when solving for the wavefunction, but in general the energy will have to be computed for each iteration and the BSH operator re-initialized with the updated $\mu$ parameter. For simple systems with a handful of electrons, the straightforward power iteration of the BSH equation is sufficient to achieve convergence, but more complicated molecular systems require additional acceleration techniques (not discussed further).

### Implementation in VAMPyR

What we need for this exercise is:

1. Define a suitable Multi-Resolution Analysis (MRA) for the problem
2. Make analytic nuclear potential and project onto the MRA
3. Make analytic initial guess for the orbital and project onto the MRA
4. Create a Helmholtz operator $\hat{G}$ using the exact value for the Hydrogen atom ($\mu=\sqrt{-2E}$, $E=-0.5 a.u.$)
5. Compute new orbital through application of the Helmholtz operator
$\tilde{\phi}^{n+1} = -2\hat{G}\left[\hat{V}\phi^n\right]$
6. Check for convergence by computing the norm of the orbital update $\Delta\phi^n = \tilde{\phi}^{n+1} - \phi^n$
7. Normalize the orbital $\phi^{n+1} = \tilde{\phi}^{n+1}/||\tilde{\phi}^{n+1}||$
8. Update orbital $\phi^{n} \leftarrow \phi^{n+1}$ for next iteration
9. Repeat steps 5-8 until your orbital has converged

## Solution

Here we put everything together in a single cell for convenience.

In [ ]:
from vampyr import vampyr3d as vp
import numpy as np
import matplotlib.pyplot as plt

# Prepare simple plotting function for visualizations
def line_plot(f, x_min, x_max):
    # Evenly spaced points along the x-axis
    r_x = np.linspace(x_min, x_max, 1000)
    data = [f([x, 0.0, 0.0]) for x in r_x]
    plt.plot(r_x, data)
    return plt

# Analytic nuclear potential
def f_nuc(r):
    Z = 1.0
    R = np.sqrt(r[0]*r[0] + r[1]*r[1] + r[2]*r[2])
    return -Z / R

# Analytic guess for solution
def f_phi(r):
    R2 = r[0]*r[0] + r[1]*r[1] + r[2]*r[2]
    return np.exp(-R2)

# Global parameters
k = 5                        # Polynomial order
L = [-20,20]                 # Simulation box size
epsilon = 1.0e-3             # Relative precision

# Define MRA and multiwavelet projector
MRA = vp.MultiResolutionAnalysis(order=k, box=L)
P_mra = vp.ScalingProjector(mra=MRA, prec=epsilon)

# Project analytic nuclear potential
V_nuc = P_mra(f_nuc)

# Initial guess for the wavefunction
phi_n = P_mra(f_phi)
phi_n.normalize()

# Prepare Helmholtz operator using exact energy
E = -0.5
mu = np.sqrt(-2*E)
G = vp.HelmholtzOperator(mra=MRA, exp=mu, prec=epsilon)
    
# Minimization loop
i = 0
thrs = 1.0e-3
update = 1.0
while (update > thrs):
    # Apply Helmholtz operator
    phi_np1 = -2*G(V_nuc * phi_n)
    
    # Compute norms for convergence check
    norm = phi_np1.norm()
    update = (phi_np1 - phi_n).norm()
    
    # Prepare for next iteration
    phi_n = phi_np1.normalize()
    
    # Plot the current wavefunction
    plt = line_plot(phi_n, -1.0, 1.0)
    
    # Print the current convergence status
    print("iteration: {}    Norm: {}   Update: {}".format(i, norm, update))
    i += 1

# Show the wavefunction plots in the same figure
plt.show()